## OpenMP

### №1
Выполнить сложение двух векторов, содержащих от 50 млн. вещественных значений (предварительно заполнить векторы случайными значениями, предусмотреть выделение и освобождение областей памяти для векторов с помощью функций стандартной библиотеки С++). Вычислить и вывести на экран время выполнения сложения, с использованием функций библиотеки OpenMP. Вывести на экран размер и любой элемент результирующего вектора.

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


В текстовой переменной `code` помещаем код реализации программы с openMP, реализованный на языке C.  
Далее записываем значение данной переменной в файл, чем формируем исходник на сервере Google Colab

In [3]:
code = """
// Программа с openMP, запускаемая через Python

// OpenMP header
#include <iostream>
#include <vector>
#include <cstdlib>
#include <ctime>
#include <omp.h>


// Функция для заполнения вектора случайными вещественными значениями
void fillVector(std::vector<double> &vec) {
    #pragma omp parallel for
    for (size_t i = 0; i < vec.size(); ++i) {
        vec[i] = static_cast<double>(rand()) / RAND_MAX;
    }
}

// Размер векторов
"""
code += 'const size_t N = {};\n'.format(50000000)
code += """
int main(int argc, char* argv[]){
    // Выделение памяти для векторов
    std::vector<double> vecA(N);
    std::vector<double> vecB(N);
    std::vector<double> vecC(N);

    // Заполнение векторов случайными значениями
    srand(static_cast<unsigned>(time(0)));
    fillVector(vecA);
    fillVector(vecB);

    // Начало измерения времени
    double startTime = omp_get_wtime();

    // Сложение векторов с использованием OpenMP
    #pragma omp parallel for
    for (size_t i = 0; i < N; ++i) {
        vecC[i] = vecA[i] + vecB[i];
    }

    // Конец измерения времени
    double endTime = omp_get_wtime();

    // Вывод результатов
    std::cout << "Время затраченное на сложение: " << (endTime - startTime) << " секунд" << std::endl;
    std::cout << "Размер получившегося вектора суммы: " << vecC.size() << std::endl;
    std::cout << "Пример элемента из вектра суммы: " << vecC[N / 2] << std::endl;

    return 0;
}
"""
# Создание исходного файла
source = open("code_1.c", "w")
source.write(code)
source.close()

Определяем число потоком для OMP

In [ ]:
%env OMP_NUM_THREADS=3

env: OMP_NUM_THREADS=3


Компилируем исходный код написанный на языке C++

In [ ]:
!g++ -o programm_1 -fopenmp code_1.c

Запускаем собранную программу

In [ ]:
!./programm_1

Время затраченное на сложение: 1.01669 секунд
Размер получившегося вектора суммы: 90000000
Пример элемента из вектра суммы: 0.718847


### №2
Выполнить задание 1, используя директиву Parallel с опцией Shared (для векторов). Сравнить полученное время с временем в задаче 1, сделать выводы.

In [ ]:
code = """
// Программа с openMP, запускаемая через Python

// OpenMP header
#include <iostream>
#include <vector>
#include <cstdlib>
#include <ctime>
#include <omp.h>


// Функция для заполнения вектора случайными вещественными значениями
void fillVector(std::vector<double> &vec) {
    #pragma omp parallel shared(vec)
    for (size_t i = 0; i < vec.size(); ++i) {
        vec[i] = static_cast<double>(rand()) / RAND_MAX;
    }
}

// Размер векторов
const size_t N = 50000000;

int main(int argc, char* argv[]){
    // Выделение памяти для векторов
    std::vector<double> vecA(N);
    std::vector<double> vecB(N);
    std::vector<double> vecC(N);

    // Заполнение векторов случайными значениями
    srand(static_cast<unsigned>(time(0)));
    fillVector(vecA);
    fillVector(vecB);

    // Начало измерения времени
    double startTime = omp_get_wtime();

    // Сложение векторов с использованием OpenMP
    #pragma omp parallel for
    for (size_t i = 0; i < N; ++i) {
        vecC[i] = vecA[i] + vecB[i];
    }

    // Конец измерения времени
    double endTime = omp_get_wtime();

    // Вывод результатов
    std::cout << "Время затраченное на сложение: " << (endTime - startTime) << " секунд" << std::endl;
    std::cout << "Размер получившегося вектора суммы: " << vecC.size() << std::endl;
    std::cout << "Пример элемента из вектра суммы: " << vecC[N / 2] << std::endl;

    return 0;
}
"""
# Создание исходного файла
source = open("code_2.c", "w")
source.write(code)
source.close()

In [ ]:
%env OMP_NUM_THREADS=3

env: OMP_NUM_THREADS=3


In [ ]:
!g++ -o programm_2 -fopenmp code_2.c

In [ ]:
!./programm_2

Время затраченное на сложение: 0.490855 секунд
Размер получившегося вектора суммы: 50000000
Пример элемента из вектра суммы: 0.550745


**Вывод:** при использовании директивы parallel с опцией shared время выполнения программы сократилось на 0.202859 секунды

### №3
Определите, какое максимальное количество нитей позволяет породить для выполнения параллельных областей программы ваша система.

In [1]:
import multiprocessing

cpu_count = multiprocessing.cpu_count()
print(f"Максимальное количество нитей: {cpu_count}")

Максимальное количество нитей: 2


### №4
При помощи трёх уровней вложенных параллельных областей породите 8 нитей (на каждом уровне параллельную область должны исполнять 2 нити). Посмотрите, как будет исполняться программа, если запретить вложенные параллельные области.

In [ ]:
code = '''
#include <stdio.h>
#include <omp.h>
#include <iostream>

int main() {
    int a = 1;
    int i;
    std::cout << "Значение 'a' до параллельной секции = " << a << std::endl;
    omp_set_num_threads(8);
    int N = 20;
    double start;

#pragma omp parallel for
    for (i = 0; i < N; i++) {
#pragma omp parallel for
        for (int j = 0; j < N; j++) {
            a++;
#pragma omp critical
            std::cout << "Увиличение значения 'a' = " << a << " нитью номер "
            << omp_get_thread_num() << std::endl;
        }
    }
    std::cout << "Значение 'a' после двух параллельных секций = " << a << std::endl;
}
'''
# Создание исходного файла
source = open("code_4.c", "w")
source.write(code)
source.close()

In [ ]:
%env OMP_NUM_THREADS=3

env: OMP_NUM_THREADS=3


In [ ]:
!g++ -o programm_4 -fopenmp code_4.c

In [ ]:
!./programm_4

Значение 'a' до параллельной секции = 1
Увиличение значения 'a' = 2 нитью номер 0
Увиличение значения 'a' = 3 нитью номер 0
Увиличение значения 'a' = 4 нитью номер 0
Увиличение значения 'a' = 5 нитью номер 0
Увиличение значения 'a' = 6 нитью номер 0
Увиличение значения 'a' = 7 нитью номер 0
Увиличение значения 'a' = 9 нитью номер 0
Увиличение значения 'a' = 10 нитью номер 0
Увиличение значения 'a' = 11 нитью номер 0
Увиличение значения 'a' = 12 нитью номер 0
Увиличение значения 'a' = 13 нитью номер 0
Увиличение значения 'a' = 14 нитью номер 0
Увиличение значения 'a' = 15 нитью номер 0
Увиличение значения 'a' = 16 нитью номер 0
Увиличение значения 'a' = 16 нитью номер 0
Увиличение значения 'a' = 17 нитью номер 0
Увиличение значения 'a' = 18 нитью номер 0
Увиличение значения 'a' = 19 нитью номер 0
Увиличение значения 'a' = 20 нитью номер 0
Увиличение значения 'a' = 21 нитью номер 0
Увиличение значения 'a' = 22 нитью номер 0
Увиличение значения 'a' = 23 нитью номер 0
Увиличение значения '

### №5
Написать пример реализации директивы For с опцией nowait, проиллюстрировать работу (печатать номер текущей итерации и номер потока, выполнившего свою часть цикла).

In [ ]:
code = '''
#include <math.h>
#include <vector>
#include <omp.h>
#include <stdio.h>
#include <iostream>


int main() {
    int i;
    int n = 10;
    int m = 10;
    std::vector<int> a, b, y, z;
    for (int i = 0; i < m; i++) {
        a.push_back(i);
        b.push_back(0);
        z.push_back(i);
        y.push_back(0);
    }
#pragma omp parallel
    {
#pragma omp for
        for (i = 1; i < n; i++) {
            std::cout << "Номер потока " << omp_get_thread_num() << " первый цикл" << std::endl;
            b[i] = int((a[i] + a[i - 1]) / 2.0);
        }
#pragma omp for
        for (i = 0; i < m; i++) {
            y[i] = int(sqrt(z[i]));
            std::cout << "Номер потока " << omp_get_thread_num() << " второй цикл" << std::endl;
        }
    }
    for (int i = 0; i < m; i++) {
        std::cout << b[i] << " ";
    }
    std::cout << std::endl;
    for (int i = 0; i < m; i++) {
        std::cout << y[i] << " ";
    }
}
'''
# Создание исходного файла
source = open("code_5.c", "w")
source.write(code)
source.close()

In [ ]:
%env OMP_NUM_THREADS=3
!g++ -o programm_5 -fopenmp code_5.c

env: OMP_NUM_THREADS=3


Запуск программы

In [ ]:
!./programm_5

Номер потока 0 первый цикл
Номер потока 0 первый цикл
Номер потока 0 первый цикл
Номер потока 2 первый цикл
Номер потока 2 первый цикл
Номер потока 2 первый цикл
Номер потока 1 первый цикл
Номер потока 1 первый цикл
Номер потока 1 первый цикл
Номер потока 0 второй цикл
Номер потока 0 второй цикл
Номер потока 0 второй цикл
Номер потока 0 второй цикл
Номер потока 2 второй цикл
Номер потока 2 второй цикл
Номер потока 2 второй цикл
Номер потока 1 второй цикл
Номер потока 1 второй цикл
Номер потока 1 второй цикл
0 0 1 2 3 4 5 6 7 8 
0 1 1 1 2 2 2 2 2 3 

### №6
Написать пример реализации директивы For с опцией collapse, проиллюстрировать работу (печатать номера текущих итераций и номер потока, их выполнившего).

In [ ]:
code = '''
#include <stdio.h>
#include <omp.h>
#include <iostream>

int main() {
    int a = 1;
    int i;
    std::cout << "Значение 'a' до параллельной секции = " << a << std::endl;
    omp_set_num_threads(8);
    int N = 20;
    double start;
    start = omp_get_wtime();

#pragma omp parallel for collapse(2)
    for (i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            a++;
#pragma omp critical
            std::cout << "Увиличение значения 'a' = " << a << " нитью номер "
            << omp_get_thread_num() << std::endl;
        }
    }
    std::cout << "Значение 'a' после двух параллельных секций = " << a << std::endl;
    std::cout << "Время выполнения = " << omp_get_wtime() - start << std::endl;
}
'''
# Создание исходного файла
source = open("code_6.c", "w")
source.write(code)
source.close()

In [ ]:
%env OMP_NUM_THREADS=3
!g++ -o programm_6 -fopenmp code_6.c

env: OMP_NUM_THREADS=3


In [ ]:
!./programm_6

Значение 'a' до параллельной секции = 1
Увиличение значения 'a' = 2 нитью номер 0
Увиличение значения 'a' = 3 нитью номер 0
Увиличение значения 'a' = 4 нитью номер 0
Увиличение значения 'a' = 5 нитью номер 0
Увиличение значения 'a' = 6 нитью номер 0
Увиличение значения 'a' = 7 нитью номер 0
Увиличение значения 'a' = 8 нитью номер 0
Увиличение значения 'a' = 9 нитью номер 0
Увиличение значения 'a' = 10 нитью номер 0
Увиличение значения 'a' = 11 нитью номер 0
Увиличение значения 'a' = 12 нитью номер 0
Увиличение значения 'a' = 13 нитью номер 0
Увиличение значения 'a' = 14 нитью номер 0
Увиличение значения 'a' = 15 нитью номер 0
Увиличение значения 'a' = 16 нитью номер 0
Увиличение значения 'a' = 17 нитью номер 0
Увиличение значения 'a' = 18 нитью номер 0
Увиличение значения 'a' = 19 нитью номер 0
Увиличение значения 'a' = 20 нитью номер 0
Увиличение значения 'a' = 21 нитью номер 0
Увиличение значения 'a' = 22 нитью номер 0
Увиличение значения 'a' = 23 нитью номер 0
Увиличение значения 'a

### №7
Написать пример реализации директивы For с опцией reduction, в котором определенным образом накапливаются значения из разных итераций цикла. Проиллюстрировать работу.

In [ ]:
code = '''
#include <iostream>
#include "omp.h"
#include <string>
#include <locale>


int main(){
	int x = 0;
	std::cout << "x в последовательной области (начало): "<< x << std::endl;
#pragma omp parallel reduction(+:x) num_threads(30)
	{
		std::cout << "Значение x в потоке (на входе): "<< x << std::endl;
		x += 1;
		std::cout << "Значение x в потоке (на выходе): " << x << std::endl;
	}
	std::cout << "x = " << x << std::endl;
	return 0;
}
'''
# Создание исходного файла
source = open("code_7.c", "w")
source.write(code)
source.close()

In [ ]:
%env OMP_NUM_THREADS=3
!g++ -o programm_7 -fopenmp code_7.c

env: OMP_NUM_THREADS=3


In [ ]:
!./programm_7

x в последовательной области (начало): 0
Значение x в потоке (на входе): 0
Значение x в потоке (на выходе): 1
Значение x в потоке (на входе): 0
Значение x в потоке (на выходе): 1
Значение x в потоке (на входе): 0
Значение x в потоке (на входе): 0
Значение x в потоке (на входе): Значение x в потоке (на входе): 0
0
Значение x в потоке (на входе): 0
Значение x в потоке (на выходе): 1
Значение x в потоке (на входе): Значение x в потоке (на входе): 0
Значение x в потоке (на выходе): 1
Значение x в потоке (на выходе): 1
0
Значение x в потоке (на выходе): 1
Значение x в потоке (на выходе): 1
Значение x в потоке (на входе): 0
Значение x в потоке (на выходе): 1
Значение x в потоке (на выходе): 1
Значение x в потоке (на входе): 0
Значение x в потоке (на выходе): 1
Значение x в потоке (на выходе): 1
Значение x в потоке (на входе): 0
Значение x в потоке (на выходе): 1
Значение x в потоке (на входе): 0
Значение x в потоке (на входе): 0Значение x в потоке (на входе): 0
Значение x в потоке (на входе)

### №8
Написать пример реализации директивы Sections и директивы Section (совместно) для 4-х задач, в которых все задачи могут работать параллельно, но 2 и 3 задачи – только вместе, друг за другом.

In [ ]:
code = '''
#include <stdio.h>
#include <omp.h>
#include <iostream>


int main() {
#pragma omp parallel sections
    {
#pragma omp section
        {
            std::cout << "T - [" << omp_get_thread_num() << "] - foo" << std::endl;
        }
#pragma omp section
        {
            std::cout << "T - [" << omp_get_thread_num() << "] - bar" << std::endl;
        }
    }// omp sections
}
'''
# Создание исходного файла
source = open("code_8.c", "w")
source.write(code)
source.close()

In [ ]:
%env OMP_NUM_THREADS=3
!g++ -o programm_8 -fopenmp code_8.c

env: OMP_NUM_THREADS=3


In [ ]:
!./programm_8

T - [0] - foo
T - [0] - bar
